In [2]:
import numpy as np

In [10]:
# --- Network model
def MSE(y,a_y):
    MSE = (1/10)*( np.sum( np.power( (y-a_y),2)))
    return MSE

def a_y(z):
    a_y = np.divide( np.exp(z), np.sum( np.exp(z)))
    return a_y

def z_y(w_y,a_2,b_y):
    w_y = np.transpose( w_y)
    b_y = np.transpose( b_y)
    z_y = np.add( np.sum(np.multiply(w_y,a_2),axis=1), b_y)
    return z_y

def a_2_nonvect(z_2):
    if z_2<0:
        z_2=0
    return z_2
a_2 = np.vectorize( a_2_nonvect)

def z_2(w_2,a_1,b_2):
    w_2 = np.transpose( w_2)
    b_2 = np.transpose( b_2)
    z_2 = np.add( np.sum(np.multiply(w_2,a_1),axis=1), b_2)
    return z_2

def a_1_nonvect(z_1):
    if z_1<0:
        z_1=0
    return z_1
a_1 = np.vectorize( a_1_nonvect)

def z_1(w_1,x,b_1):
    w_1 = np.transpose( w_1)
    b_1 = np.transpose( b_1)
    z_1 = np.add( np.sum(np.multiply(w_1,x),axis=1), b_1)
    return z_1

def init_params():
    w_1 = np.random.rand(784,1)
    b_1 = np.random.rand(784,1)
    w_2 = np.random.rand(10,1)
    b_2 = np.random.rand(10,1)
    w_y = np.random.rand(10,1)
    b_y = np.random.rand(10,1)

def run_model(x,y,w_1,b_1,w_2,b_2,w_y,b_y):
    z_1 = z_1(w_1,x,b_1)
    a_1 = a_1(z_1)
    z_2 = z_2(w_2,a_1,b_2)
    a_2 = a_2(z_2)
    z_y = z_y(w_y,a_2,b_y)
    a_y = a_y(z_y)
    return( a_1, a_2, z_1, z_2, w_2, w_y, a_y)

# --- Gradient descend
def indicator_nonvect(z):
    if z>0:
        z=1
    else:
        z=0
    return z
indicator = np.vectorize( indicator_nonvect)

# Back prop for a single sample
def single_back_prop(x, y, a_1, a_2, z_1, z_2, w_2, w_y, a_y):
    DZ = np.array([[]])
    if (DZ.shape == (1, 0)):
        DZ = np.sum( a_y*a_y[0]) - (a_y[0]*a_y[0])[0] + (a_y[0]*( 1-a_y[0]))[0]
        i=1
    for i in range(a_y.shape[0]):
        DZ = np.append( ( DZ, np.sum( a_y*a_y[i]) - (a_y[i]*a_y[0])[0] + (a_y[i]*( 1-a_y[i]))[0]), axis=1)

    S_1 = np.multiply( ((-(y-a_y))/5), DZ)
    S_2 = np.multiply( np.multiply(S_1, w_y), indicator(z_2))
    S_3 = np.multiply( np.multiply( np.multiply( S_1,S_2), w_2), indicator(z_1))

    sgrad_w_1 = np.multiply( np.multiply( np.multiply( S_1, S_2), S3), x)
    sgrad_b_1 = np.multiply( np.multiply( S_1, S_2), S3)
    sgrad_w_2 = np.multiply( np.multiply( S_1, S_2), a_1)
    sgrad_b_2 = np.multiply( S_1, S_2)
    sgrad_w_y = np.multiply( S_1, a_2)
    sgrad_b_y = S_1
    sgrad = np.array([sgrad_w_1,sgrad_b_1,sgrad_w_2,sgrad_b_2,sgrad_w_y,sgrad_b_y])
    return( sgrad)

# Averaged back prop for supposedly randomly ordered batches of samples
def stochastic_back_prop(K, x, y, a_1, a_2, z_1, z_2, w_2, w_y, a_y):
    batch=0
    # defining batch
    for i in range(K):
        x = x[(i+batch):(i+batch+K),:]
        y = y[(i+batch):(i+batch+K)]
        #passing batch to back_prop
        if (bgrad.shape == (1, 0)):
            bgrad = single_back_prop(x[j,:], y[j], a_1, a_2, z_1, z_2, w_2, w_y, a_y)
            j=1
        for j in range( y.shape[0]):
            bgrad = np.append( (bgrad, single_back_prop(x[j,:], y[j], a_1, a_2, z_1, z_2, w_2, w_y, a_y)), axis=1)
        bgrad = np.divide( np.sum( bgrad, axis=0),K)
        batch=batch+1    
    return( bgrad)

# Optimization with a given batch
def stochastic_grad_descend( epsilon, bgrad_w_1, bgrad_b_1, bgrad_w_2, bgrad_b_2, bgrad_w_y, bgrad_b_y):
    w_1 = np.subtract( w_1, np.multiply(epsilon, bgrad_w_1))
    b_1 = np.subtract( b_1, np.multiply(epsilon, bgrad_b_1))
    w_2 = np.subtract( w_2, np.multiply(epsilon, bgrad_w_2))
    b_2 = np.subtract( b_2, np.multiply(epsilon, bgrad_b_2))
    w_y = np.subtract( w_y, np.multiply(epsilon, bgrad_w_y))
    b_y = np.subtract( b_y, np.multiply(epsilon, bgrad_b_y))
    return( w_1, b_1, w_2, b_2, w_y, b_y)